In [1]:
import xarray as xr
import rioxarray
import rasterio
import numpy as np
import dask
from dask.distributed import Client
#client = Client()
#client
dask.config.set(scheduler="synchronous")

In [2]:
import os
if 'notebooks' in os.getcwd():
    os.chdir("..")
    import sys; sys.path.append(os.getcwd())

CHUNK_SCHEMA = dict(time=27, x=512, y=512)

Read in all the data types

In [3]:
def preprocess_mortality(ds):
    year = os.path.basename(ds.encoding["source"])
    year = int(year.replace(".tif", ""))
    return ds.squeeze(drop=True)\
        .drop_vars("spatial_ref")\
        .rename(band_data="mortality")\
        .assign_coords(time=year)

damage = xr.open_mfdataset(
    "data_working/damage_rasters/*.tif",
    preprocess=preprocess_mortality,
    concat_dim="time", 
    combine="nested"
).chunk(**CHUNK_SCHEMA)

In [4]:
daymet = xr.open_mfdataset('data_working/daymet/*.nc', concat_dim="time", combine="nested")\
    .drop_vars("spatial_ref")\
    .chunk(**CHUNK_SCHEMA)
daymet

<xarray.Dataset> Size: 3GB
Dimensions:  (time: 27, y: 3251, x: 2741)
Coordinates:
  * time     (time) int64 216B 1997 1998 1999 2000 2001 ... 2020 2021 2022 2023
  * x        (x) float64 22kB -1.424e+07 -1.424e+07 ... -1.15e+07 -1.15e+07
  * y        (y) float64 26kB 6.674e+06 6.673e+06 ... 3.425e+06 3.424e+06
Data variables:
    prcp     (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    vp       (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    tmin     (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [5]:
def preprocess_fire(ds):
    year = os.path.basename(ds.encoding["source"])
    year = int(year.replace(".tif", ""))
    return ds.squeeze(drop=True)\
        .drop_vars("spatial_ref")\
        .rename(band_data="fire")\
        .assign_coords(time=year)

fire = xr.open_mfdataset(
    "data_working/mtbs_rasters/*.tif",
    preprocess=preprocess_fire,
    concat_dim="time", combine="nested"
).chunk(**CHUNK_SCHEMA)

In [6]:
fire

<xarray.Dataset> Size: 927MB
Dimensions:  (time: 26, y: 3251, x: 2741)
Coordinates:
  * x        (x) float64 22kB -1.424e+07 -1.424e+07 ... -1.15e+07 -1.15e+07
  * y        (y) float64 26kB 6.674e+06 6.673e+06 ... 3.425e+06 3.424e+06
  * time     (time) int64 208B 1997 1998 1999 2000 2001 ... 2019 2020 2021 2022
Data variables:
    fire     (time, y, x) float32 927MB dask.array<chunksize=(26, 512, 512), meta=np.ndarray>

In [7]:
def preprocess_spp_ba(ds):
    spp = os.path.basename(ds.encoding["source"])\
        .replace(".tif", "")

    return ds.squeeze(drop=True).rename(band_data=spp)

spp_ba = xr.open_mfdataset(
    "data_working/genus_basal_area/*.tif", 
    preprocess=preprocess_spp_ba,
    combine="nested"
).drop_vars("spatial_ref").expand_dims(time=damage.time.shape[0]).assign_coords(time=damage.time).chunk(**CHUNK_SCHEMA)

In [8]:
spp_ba

<xarray.Dataset> Size: 6GB
Dimensions:      (x: 2741, y: 3251, time: 27)
Coordinates:
  * x            (x) float64 22kB -1.424e+07 -1.424e+07 ... -1.15e+07 -1.15e+07
  * y            (y) float64 26kB 6.674e+06 6.673e+06 ... 3.425e+06 3.424e+06
  * time         (time) int64 216B 1997 1998 1999 2000 ... 2020 2021 2022 2023
Data variables:
    abies        (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    picea        (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    pinus        (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    populus      (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    pseudotsuga  (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    tsuga        (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [9]:
terrain = xr.open_dataset("data_working/terrain.nc")\
    .drop_vars(["spatial_ref", "band"])\
    .expand_dims(time=damage.time.shape[0])\
    .assign_coords(time=damage.time)\
    .chunk(**CHUNK_SCHEMA)

terrain

<xarray.Dataset> Size: 4GB
Dimensions:    (x: 2741, y: 3251, time: 27)
Coordinates:
  * x          (x) float64 22kB -1.424e+07 -1.424e+07 ... -1.15e+07 -1.15e+07
  * y          (y) float64 26kB 6.674e+06 6.673e+06 ... 3.425e+06 3.424e+06
  * time       (time) int64 216B 1997 1998 1999 2000 ... 2020 2021 2022 2023
Data variables:
    elev       (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    slope      (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    northness  (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    eastness   (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [10]:
vod = xr.open_dataset("data_working/summer_vod.nc")\
    .drop_vars("spatial_ref")\
    .chunk(**CHUNK_SCHEMA)
vod = vod.assign_coords(time=vod.time.dt.year)
vod

<xarray.Dataset> Size: 1GB
Dimensions:    (x: 2741, y: 3251, time: 21)
Coordinates:
  * x          (x) float64 22kB -1.424e+07 -1.424e+07 ... -1.15e+07 -1.15e+07
  * y          (y) float64 26kB 6.674e+06 6.673e+06 ... 3.425e+06 3.424e+06
  * time       (time) int64 168B 1998 1999 2000 2001 ... 2015 2016 2017 2018
Data variables:
    vod        (time, y, x) float32 749MB dask.array<chunksize=(21, 512, 512), meta=np.ndarray>
    delta_vod  (time, y, x) float32 749MB dask.array<chunksize=(21, 512, 512), meta=np.ndarray>
Attributes: (12/29)
    calendar:                  standard
    geospatialverticalmin:     0.0
    timecoverageresolution:    P1D
    DOI:                       10.5281/zenodo.2575599
    product_version:           01.0
    geospatialverticalmax:     0.0
    ...                        ...
    creatorurl:                http://rs.geo.tuwien.ac.at
    license:                   Creative Commons Attribution 4.0 International
    creatorname:               Department of Geodesy and Geoinformation, Tech...
    timecoveragestartdate:     1997-12-01T00:00:00
    contact:                   vodca@geo.tuwien.ac.at
    spatialresolution:         25 km

Merge everybody together. Note this is very fast because all the arrays are on dask and have the same chunking strategy.

In [11]:
all_data = xr.combine_by_coords(
    [vod, terrain, spp_ba, fire, daymet, damage],
    coords="minimal",
    compat="override",
    combine_attrs="drop"
).drop_vars(["delta_vod"])

# Shift so we are predicting *next year's* mortality in each sample
all_data["mortality"] = all_data["mortality"].shift(time=-1)

all_data = all_data.sel(time=slice(1998, 2017))

Construct a stacked dataset, rolling will happen just before write so we can do all the windows at once.

In [12]:
all_stack = all_data.to_stacked_array(new_dim="band", sample_dims=["time", "x", "y"])

In [13]:
all_stack

<xarray.DataArray 'abies' (time: 20, y: 3251, x: 2741, band: 16)> Size: 11GB
dask.array<concatenate, shape=(20, 3251, 2741, 16), dtype=float32, chunksize=(20, 512, 512, 1), chunktype=numpy.ndarray>
Coordinates:
  * x         (x) float64 22kB -1.424e+07 -1.424e+07 ... -1.15e+07 -1.15e+07
  * y         (y) float64 26kB 6.674e+06 6.673e+06 ... 3.425e+06 3.424e+06
  * time      (time) int64 160B 1998 1999 2000 2001 2002 ... 2014 2015 2016 2017
  * band      (band) object 128B MultiIndex
  * variable  (band) <U11 704B 'abies' 'picea' 'pinus' ... 'prcp' 'vp' 'tmin'

Write data as zarr arrays.

In [33]:
def enumerate_xy_chunks(array):
    steps_x = array.chunks[2]
    steps_y = array.chunks[1]

    upto_x = 0
    for sx in steps_x:
        upto_y = 0
        for sy in steps_y:
            chunk = array.isel(
                x=slice(upto_x, upto_x + sx),
                y=slice(upto_y, upto_y + sy)
            )
            upto_y += sy
            yield chunk
            
        upto_x += sx

In [34]:
# 2-10 years. Based on initial experiment with CA dataset, the temporal
# benefit plateaus around 5 years. Could be better with this larger
# dataset so we will test a range of values.
import zarr
from google.cloud import storage

client = storage.Client(project="forest-lst")
bucket = client.bucket("ads_training_data")

time_windows = list(range(2, 11))

def write_windowed_data(chunk):
    for tw in time_windows:
        print(f"{tw}...", end="")
        zarr_chunk_size = (200_000, 16, tw)
        # print("Now writing", tw)
        out = f"gs://ads_training_data/windowed{tw}.zarr"
        
        #print("Windowing...")
        chunk_rolling = chunk.rolling(time=tw).construct(time="time_window")\
            .stack(sample=["time", "x", "y"])\
            .transpose("sample", ...)

        #print("Slicing non-na samples...")
        nona_idx = np.where(chunk_rolling.notnull().all(dim=["band", "time_window"]))[0]
        chunk_nona = chunk_rolling.isel(sample=nona_idx)
        #print("No-na shape", chunk_nona.values.shape)
        # Since the time window is increasing, if there are no samples in this
        # window, we can safely skip all remaining windows.
        if not chunk_nona.values.shape[0]:
            print("all-NA", end="")
            break

        #print("Writing to store...")
        if storage.Blob(bucket=bucket, name=f"windowed{tw}.zarr/.zarray").exists(client):
            # If the store already exists, append to the first axis.
            store = zarr.creation.open_array(out)
            store.append(chunk_nona.values, axis=0)
        else:
            # Otherwise, make a store that has the shape of the first
            # chunk we are writing.
            store = zarr.creation.open_array(
                out, chunks=zarr_chunk_size, shape=chunk_nona.values.shape,
                cache_metadata=False, cache_attrs=False, mode="w" 
            )
            store[::] = chunk_nona.values

In [ ]:
counter = 0
for chunk in enumerate_xy_chunks(all_stack):
    counter += 1
    print("Writing chunk", counter, ":", end="")
    chunk = chunk.compute()
    write_windowed_data(chunk)
    print()

Writing chunk 1 :2...3...4...5...6...7...8...9...10...
Writing chunk 2 :2...
3...
4...
5...
6...
7...
8...
9...
10...

Writing chunk 3 :2...
3...
4...
5...
6...
7...
8...
9...
10...

Writing chunk 4 :2...
3...
4...
5...
6...
7...
8...
9...
10...

Writing chunk 5 :2...3...4...5...6...7...8...9...10...
Writing chunk 6 :2...3...4...5...6...7...8...9...10...
Writing chunk 7 :2...3...4...5...6...7...8...9...10...
Writing chunk 8 :2...
3...
4...
5...
6...
7...
8...
9...
10...

Writing chunk 9 :2...
3...
4...
5...
6...
7...
8...
9...
10...

Writing chunk 10 :2...
3...
4...
5...
6...